In [1]:
import requests
import bs4
from statistics import mean
from bs4 import BeautifulSoup as BS
import pandas as pd
import time
import re
import string
from nltk import tokenize
from selenium import webdriver
import os
from textblob import TextBlob
from google_trans_new import google_translator
translator = google_translator()
from collections import defaultdict
from pandas.core.common import flatten
from collections import Counter
import nltk
import numpy as np
import short_url
import ast
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity
from scipy import spatial

In [2]:
%run EDSF_class.ipynb

In [3]:
stemmer = nltk.stem.porter.PorterStemmer()
remove_punctuation_map = dict((ord(char), None) for char in string.punctuation)

In [4]:
def stem_tokens(tokens):
    return [stemmer.stem(item) for item in tokens]

In [5]:
def normalize(text):
    return stem_tokens(nltk.word_tokenize(text.lower().translate(remove_punctuation_map)))

In [6]:
df = pd.read_csv('dataframe_data steward.csv')

In [7]:
df

,id,title,company,link,description
0,DSP_0,Data steward / analyst,JLL,https://nl.indeed.com/rc/clk?jk=32d6b156cbb9d2...,"<div class=""jobsearch-JobComponent-description..."
1,DSP_1,Global Process Director,Cargill,https://nl.indeed.com/rc/clk?jk=f468bb81155e0a...,"<div class=""jobsearch-JobComponent-description..."
2,DSP_2,Human Resources Generalist,THE RENEWAL WORKSHOP,https://nl.indeed.com/rc/clk?jk=cb367910062ed1...,"<div class=""jobsearch-JobComponent-description..."
3,DSP_3,Data Engineer,Optiver,https://nl.indeed.com/rc/clk?jk=1bdc22b275c924...,"<div class=""jobsearch-JobComponent-description..."
4,DSP_4,Senior Data Management Analyst Mortgages,ING,https://nl.indeed.com/rc/clk?jk=6ac940b5c663f1...,"<div class=""jobsearch-JobComponent-description..."
5,DSP_5,Product Owner Data Management Mortgages,ING,https://nl.indeed.com/rc/clk?jk=deca9994e792aa...,"<div class=""jobsearch-JobComponent-description..."
6,DSP_6,RA Manager,RB,https://nl.indeed.com/rc/clk?jk=76a34a12579b88...,"<div class=""jobsearch-JobComponent-description..."
7,DSP_7,Data Engineer,Carrierecafe,https://nl.indeed.com/rc/clk?jk=f70cb6470c9974...,"<div class=""jobsearch-JobComponent-description..."
8,DSP_8,Inbound Marketer,24sessions,https://nl.indeed.com/rc/clk?jk=8376556adb3173...,"<div class=""jobsearch-JobComponent-description..."
9,DSP_9,Data Analist,Carrierecafe,https://nl.indeed.com/rc/clk?jk=766d1bdeda95fb...,"<div class=""jobsearch-JobComponent-description..."


In [8]:
def translate(text):
    return translator.translate(text, lang_tgt='en').lower()

In [9]:
def extract_text(raw_html):
    clean_text = []
    for description in raw_html:
        parsed = BS(description, "html.parser")
        text = parsed.text.replace('\n', ' ')
        if len(text) > 5000:
            text = text[:4999]
        lang = TextBlob(text[100:200]).detect_language()
        if lang == 'nl':
            text = translate(text)
        clean_text.append(text)
    return clean_text

In [10]:
corpus = extract_text(df['description'])

In [11]:
def similarity(vectorizer, docs_tfidf, query):
    query_tfidf = vectorizer.transform(query)
    cosinesim = cosine_similarity(docs_tfidf, query_tfidf, dense_output=False)
    return cosinesim

In [12]:
def similarity_dict(corpus, knowledge_units):
    similarity_dict = {}
    titles = np.array(knowledge_units)[:,0]
    words = np.array(knowledge_units)[:,1]
    vectorizer = TfidfVectorizer(tokenizer=normalize, stop_words='english')
    docs_tfidf = vectorizer.fit_transform(corpus)
    sim = similarity(vectorizer, docs_tfidf, words)     
    
    for i in range(sim.shape[0]):
        d = sim.getrow(i)
        idx = d.sorted_indices().indices
        data = d.sorted_indices().data
        similarity_dict[df['id'][i]] = [[titles[a], b] for a, b in zip(idx, data)]
    
    return similarity_dict

In [13]:
d = similarity_dict(corpus, kagcode_with_ka)
d

C:\Users\thomh\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'afterward', 'alon', 'alreadi', 'alway', 'ani', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becaus', 'becom', 'befor', 'besid', 'cri', 'describ', 'dure', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'formerli', 'forti', 'ha', 'henc', 'hereaft', 'herebi', 'hi', 'howev', 'hundr', 'inde', 'latterli', 'mani', 'meanwhil', 'moreov', 'mostli', 'nobodi', 'noon', 'noth', 'nowher', 'onc', 'onli', 'otherwis', 'ourselv', 'perhap', 'pleas', 'seriou', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 'someth', 'sometim', 'somewher', 'themselv', 'thenc', 'thereaft', 'therebi', 'therefor', 'thi', 'thu', 'togeth', 'twelv', 'twenti', 'veri', 'wa', 'whatev', 'whenc', 'whenev', 'wherea', 'whereaft', 'wherebi', 'wherev', 'whi', 'yourselv'] not in stop_words.
  'stop

{'DSP_0': [['KA01.01', 0.08820673406632133],
  ['KA01.03', 0.11968359751229656],
  ['KA01.04 ', 0.14844960287155406],
  ['KA01.05', 0.04049126468401141],
  ['KA01.06', 0.018899450300858052],
  ['KA02.01', 0.11062589945639856],
  ['KA02.02', 0.2109949070075046],
  ['KA02.03', 0.03789693311069505],
  ['KA02.04', 0.2248438305238013],
  ['KA02.05', 0.2141413735209279],
  ['KA02.06', 0.19602772167610588],
  ['KA02.07', 0.17027740256445584],
  ['KA03.01', 0.26864174734246604],
  ['KA03.02', 0.2100848866356385],
  ['KA03.03', 0.3355421890866012],
  ['KA03.04', 0.335235530979099],
  ['KA03.05', 0.17357621482338326],
  ['KA03.06', 0.06130884608065972],
  ['KA04.01', 0.13513891105288903],
  ['KA04.02', 0.02716646997702588],
  ['KA05.01', 0.19802300120658428],
  ['KA05.02', 0.06794431609318145]],
 'DSP_1': [['KA01.01', 0.0307388777472397],
  ['KA01.02', 0.05264725384382529],
  ['KA01.03', 0.04700286501931693],
  ['KA01.04 ', 0.07018123928809918],
  ['KA01.05', 0.02064235060049301],
  ['KA01.06', 

In [14]:
#%store d